In [25]:
import warnings
from typing import Callable
import time
from functools import wraps
from loguru import logger

In [26]:
def IgnoreWarnings(func: Callable) -> Callable:
    """
    Декоратор для игнорирования предупреждений RuntimeWarning и UserWarning

    Args:
        func (Callable): декорируемая функция

    Returns:
        Callable: декорируемая функция
    """

    @wraps(func)
    def Wrapper(*args, **kwargs):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", RuntimeWarning)
            warnings.simplefilter("ignore", UserWarning)

            return func(*args, **kwargs)

    return Wrapper


def Retry(attempts_amount=3, exception_to_check=Exception, sleep_time=1) -> Callable:
    """
    Декоратор для повторения попыток выполнения функции

    Args:
        attempts_amount (int, optional): кол-во попыток. Defaults to 3.
        exception_to_check (_type_, optional): улавливаемое исключение. Defaults to Exception.
        sleep_time (int, optional): время ожидания между попытками. Defaults to 3.

    Returns:
        Callable: декорируемая функция
    """

    def Decorate(func: Callable) -> Callable:
        @wraps(func)
        def Wrapper(*args, **kwargs):
            for attempt in range(1, attempts_amount+1):
                try:
                    return func(*args, **kwargs)

                except exception_to_check as exception:
                    logger.warning(f"{exception} | Attempt: {
                                   attempt}".ljust(77))
                    if attempt < attempts_amount:
                        time.sleep(sleep_time)

            logger.error("All attempts failed".ljust(77))
            return None
        return Wrapper
    return Decorate


In [27]:
@Retry()
def my_function(x):
    if x == 0:
        raise ZeroDivisionError("Деление на ноль!")
    return x / 2

result = my_function(0)
print(result)

2024-10-14 12:53:46.959 | WARNING  | __main__:Wrapper:44 - Деление на ноль! | Attempt: 1                                                
2024-10-14 12:53:47.968 | WARNING  | __main__:Wrapper:44 - Деление на ноль! | Attempt: 2                                                
2024-10-14 12:53:48.970 | WARNING  | __main__:Wrapper:44 - Деление на ноль! | Attempt: 3                                                
2024-10-14 12:53:48.970 | ERROR    | __main__:Wrapper:49 - All attempts failed                                                          


None
